In [ ]:
# %%
import os
if True:
    os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# import torch
from train_consistency_unlearning import train_main, parse_args
from utils.utils import dict_to_argv

concepts = [
    ('cat', 1200),
    ('apple', 1200),
    ('van_gogh', 1200),
    ('claude_monet', 1200),
    ('elon_musk', 1500),
    ('taylor_swift', 1500),
    ('naked', 1500),
]


def get_val_prompts(concept: str):
    validation_prompt = f"a photo of {concept}"
    if concept == 'naked':
        validation_prompt = "a photo of naked person"
    if concept == 'van_gogh':
        validation_prompt = "a painting in the style of Van Gogh"
    if concept == 'claude_monet':
        validation_prompt = "a painting in the style of Claude Monet"
    if concept == 'elon_musk':
        validation_prompt = "a photo of Elon Musk"
    if concept == 'taylor_swift':
        validation_prompt = "a photo of Taylor Swift"
    return validation_prompt


method = "unlearning-consistency"
for concept, steps in concepts:
    print(f"↓=============开始遗忘{concept}=============↓")
    params = {
        "pretrained_model_name_or_path": "stable-diffusion-v1-5/stable-diffusion-v1-5",
        "variant": "fp16",
        "unlearn_data_dirs": f"./data/{concept};",
        "unlearn_concepts": f"{concept};",
        "retain_data_dir": "./data/other",
        "unlearn_batch_size": 5,
        "retain_batch_size": 5,
        "unlearn_data_copy_multiple": 5,
        "output_dir": f"./outputs/unlearning-consistency-model-{concept}",
        "resolution": 512,
        "alpha": 0.2,  # intermediate loss (unlearn and retain)
        "beta": 0.2,  # ground truth loss (unlearn and retain)
        "gamma": 0.0,  # ground truth loss and intermediate loss
        "num_train_epochs": 1,  # 一般取1
        "max_train_steps": steps,
        "learning_rate": 2e-5,
        "lr_scheduler": "constant",
        # "lr_warmup_steps": 50,
        "validation_prompts": f"{get_val_prompts(concept)}; a photo of woman",
        "validation_steps": 100,
        "checkpointing_steps": 100,
        "num_validation_images": 6,
        "train_text_encoder": False,
        "rank": 8,
        "seed": 100,
        "report_to": "wandb",
        "wandb_mode": "offline",
        "wandb_run_name": f"{method}",
        "mixed_precision": "fp16",
    }
    args = parse_args(dict_to_argv(params))
    train_main(args)
    print(f"↑=============结束遗忘{concept}=============↑\n\n")
    # torch.cuda.empty_cache()